<a href="https://colab.research.google.com/github/tariqulq-cyber/Pemrosesan-teks-praktek/blob/main/tugas1_pemrosesan_teks_teori_scraping_%26_casefolding_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install google-play-scraper pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.4 MB/s eta 0:00:00


SCREPING

In [24]:
import pandas as pd
from google_play_scraper import Sort, reviews
from datetime import datetime
import os
import textwrap # Import modul untuk memecah teks panjang agar tampilan rapi

# --- 1. KONFIGURASI DAN PARAMETER ---

APP_ID = 'com.rtsoft.growtopia'
LIMIT_KOMENTAR = 1000
TAHUN_BATAS_ATAS = 2025 # Batas tahun yang diminta: 2025 ke bawah
NAMA_FILE_OUTPUT = 'komentar_growtopia_2025_kebawah.csv'

# Batas tanggal eksklusif (1 Januari tahun berikutnya)
# Ulasan harus <= 31 Desember 2025
TANGGAL_BATAS = datetime(TAHUN_BATAS_ATAS + 1, 1, 1)

print("=======================================================")
print(f"TARGET: Scraping {LIMIT_KOMENTAR} komentar terbaru dari Growtopia")
print(f"FILTER: Komentar dengan tanggal <= 31 Desember {TAHUN_BATAS_ATAS}")
print("=======================================================")

# --- 2. PROSES SCRAPING (PENGAMBILAN DATA) ---

try:
    # Mengambil ulasan (review) terbaru
    # Ini adalah satu-satunya proses pengambilan data/scraping.
    result, continuation_token = reviews(
        APP_ID,
        lang='id',    # Bahasa Indonesia
        country='id', # Negara Indonesia
        sort=Sort.NEWEST, # Mengurutkan dari yang terbaru
        count=LIMIT_KOMENTAR,
        filter_score_with=None # Mengambil semua skor rating
    )

    # Mengubah hasil scraping menjadi DataFrame Pandas
    df_raw = pd.DataFrame(result)

    print(f"\n[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: {len(df_raw)}.")

except Exception as e:
    print(f"\n[ERROR] Terjadi kesalahan saat scraping: {e}")
    df_raw = pd.DataFrame()


# --- 3. PROSES FILTERING BERDASARKAN TANGGAL ---

df_scraped_filtered = pd.DataFrame()

if not df_raw.empty:

    # 3.1. Mempersiapkan kolom tanggal
    df_raw['at'] = pd.to_datetime(df_raw['at'])

    # 3.2. Menerapkan Filtering
    df_scraped_filtered = df_raw[df_raw['at'] < TANGGAL_BATAS].copy()

    # 3.3. Memilih dan menata kolom untuk output
    df_scraped_filtered = df_scraped_filtered[[
        'userName',
        'at',
        'content',
        'score',
        'thumbsUpCount'
    ]].reset_index(drop=True)

    # Menambahkan kolom NO dan mengubah format tanggal
    df_scraped_filtered.insert(0, 'NO', range(1, 1 + len(df_scraped_filtered)))
    df_scraped_filtered['at'] = df_scraped_filtered['at'].dt.strftime('%Y-%m-%d %H:%M:%S')

    print(f"[INFO] Setelah filter tahun {TAHUN_BATAS_ATAS} ke bawah, sisa komentar: {len(df_scraped_filtered)}.")
    if len(df_scraped_filtered) == 0:
        print("[PERINGATAN] Tidak ada komentar dalam 1000 ulasan terbaru yang berasal dari tahun tersebut.")

else:
    print("\n[ERROR] Data scraping kosong, tidak dapat melakukan filtering.")


# --- 4. TAMPILAN OUTPUT LENGKAP (RATA KIRI DAN RAPI - TANPA CASEFOLDING) ---

if not df_scraped_filtered.empty:

    pd.reset_option('all')

    # --- Penyiapan Format Tabel Manual untuk Rata Kiri ---

    # Mendefinisikan lebar kolom:
    WIDTH_NO = 4
    WIDTH_USER = 20
    WIDTH_DATE = 20
    WIDTH_SCORE = 5
    WIDTH_THUMBS = 5

    # Menghitung lebar kolom konten untuk wrapping
    WIDTH_NON_CONTENT = WIDTH_NO + WIDTH_USER + WIDTH_DATE + WIDTH_SCORE + WIDTH_THUMBS + 10
    WIDTH_CONTENT_WRAP = 120 - WIDTH_NON_CONTENT - 4

    # Header Tabel (Diperbarui untuk mencerminkan bahwa konten adalah MENTAH)
    header = (
        f"{'NO':<{WIDTH_NO}} | {'USERNAME':<{WIDTH_USER}} | {'AT':<{WIDTH_DATE}} | "
        f"{'CONTENT (Full, RAW TEXT)':<{WIDTH_CONTENT_WRAP}} | {'SCORE':<{WIDTH_SCORE}} | {'SUKA':<{WIDTH_THUMBS}} |"
    )
    separator = "=" * len(header)

    # --- Mencetak Output ke Konsol ---

    print("\n\n=======================================================")
    print(f"HASIL AKHIR SCRAPING (Total Komentar Tampil: {len(df_scraped_filtered)})")
    print("=======================================================")
    print("CATATAN: Komentar LENGKAP ditampilkan sebagai Teks Mentah (RAW TEXT), sesuai aslinya.")
    print("-------------------------------------------------------------------")

    # Cetak Header dan Separator
    print(header)
    print(separator)

    # Cetak Isi Tabel (Looping dan Text Wrapping)
    for index, row in df_scraped_filtered.iterrows():
        # *** PERUBAHAN UTAMA: MENGGUNAKAN KONTEN MENTAH (TANPA .lower()) ***
        raw_content = str(row['content']) # Mengambil konten mentah

        # 2. Pecah konten panjang menjadi list baris
        wrapped_lines = textwrap.wrap(raw_content, width=WIDTH_CONTENT_WRAP)

        if not wrapped_lines:
            wrapped_lines = ["-"]

        # --- Cetak Baris Pertama (dengan semua data) ---
        line = (
            f"{row['NO']:<{WIDTH_NO}} | {row['userName']:<{WIDTH_USER}} | {row['at']:<{WIDTH_DATE}} | "
            f"{wrapped_lines[0]:<{WIDTH_CONTENT_WRAP}} | {row['score']:<{WIDTH_SCORE}} | {row['thumbsUpCount']:<{WIDTH_THUMBS}} |"
        )
        print(line)

        # --- Cetak Baris Lanjutan (dengan indentasi) ---
        for subsequent_line in wrapped_lines[1:]:

            # Kolom Non-Content diganti dengan spasi kosong
            blank_columns = (
                f"{'':<{WIDTH_NO}} | {'':<{WIDTH_USER}} | {'':<{WIDTH_DATE}} | "
            )

            # Baris lanjutan hanya berisi indentasi, konten, dan spasi kosong di kolom akhir
            continuation_line = (
                f"{blank_columns}{subsequent_line:<{WIDTH_CONTENT_WRAP}} | {'':<{WIDTH_SCORE}} | {'':<{WIDTH_THUMBS}} |"
            )
            print(continuation_line)

    print(separator)


TARGET: Scraping 1000 komentar terbaru dari Growtopia
FILTER: Komentar dengan tanggal <= 31 Desember 2025

[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: 1000.
[INFO] Setelah filter tahun 2025 ke bawah, sisa komentar: 1000.


HASIL AKHIR SCRAPING (Total Komentar Tampil: 1000)
CATATAN: Komentar LENGKAP ditampilkan sebagai Teks Mentah (RAW TEXT), sesuai aslinya.
-------------------------------------------------------------------
NO   | USERNAME             | AT                   | CONTENT (Full, RAW TEXT)                             | SCORE | SUKA  |
1    | RE 8D                | 2025-10-03 08:56:35  | Bad game, hard to log in, makes you lazy to play but | 3     | 0     |
     |                      |                      | if you want to play you definitely can't log in, no  |       |       |
     |                      |                      | idea what error it is, even though you've never used |       |       |
     |                      |                      | bo

/tmp/ipython-input-2822102058.py:84: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/tmp/ipython-input-2822102058.py:84: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


CSEFOLDING


In [25]:
import pandas as pd
from google_play_scraper import Sort, reviews
from datetime import datetime
import os
import textwrap # Import modul untuk memecah teks panjang

# --- 1. KONFIGURASI DAN PARAMETER ---

APP_ID = 'com.rtsoft.growtopia'
LIMIT_KOMENTAR = 1000
TAHUN_BATAS_ATAS = 2025 # Batas tahun yang diminta: 2025 ke bawah
NAMA_FILE_OUTPUT = 'komentar_growtopia_2025_kebawah.csv'

# Batas tanggal eksklusif (1 Januari tahun berikutnya)
# Ulasan harus <= 31 Desember 2025
TANGGAL_BATAS = datetime(TAHUN_BATAS_ATAS + 1, 1, 1)

print("=======================================================")
print(f"TARGET: Scraping {LIMIT_KOMENTAR} komentar terbaru dari Growtopia")
print(f"FILTER: Komentar dengan tanggal <= 31 Desember {TAHUN_BATAS_ATAS}")
print("=======================================================")

# --- 2. PROSES SCRAPING (PENGAMBILAN DATA) ---

try:
    # Mengambil ulasan (review) terbaru
    result, continuation_token = reviews(
        APP_ID,
        lang='id',    # Bahasa Indonesia
        country='id', # Negara Indonesia
        sort=Sort.NEWEST, # Mengurutkan dari yang terbaru
        count=LIMIT_KOMENTAR,
        filter_score_with=None # Mengambil semua skor rating
    )

    # Mengubah hasil scraping menjadi DataFrame Pandas
    df_raw = pd.DataFrame(result)

    print(f"\n[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: {len(df_raw)}.")

except Exception as e:
    print(f"\n[ERROR] Terjadi kesalahan saat scraping: {e}")
    df_raw = pd.DataFrame()


# --- 3. PROSES FILTERING BERDASARKAN TANGGAL ---

df_scraped_filtered = pd.DataFrame()

if not df_raw.empty:

    # 3.1. Mempersiapkan kolom tanggal
    df_raw['at'] = pd.to_datetime(df_raw['at'])

    # 3.2. Menerapkan Filtering
    df_scraped_filtered = df_raw[df_raw['at'] < TANGGAL_BATAS].copy()

    # 3.3. Memilih dan menata kolom untuk output
    df_scraped_filtered = df_scraped_filtered[[
        'userName',
        'at',
        'content',
        'score',
        'thumbsUpCount'
    ]].reset_index(drop=True)

    # Menambahkan kolom NO dan mengubah format tanggal
    df_scraped_filtered.insert(0, 'NO', range(1, 1 + len(df_scraped_filtered)))
    df_scraped_filtered['at'] = df_scraped_filtered['at'].dt.strftime('%Y-%m-%d %H:%M:%S')

    print(f"[INFO] Setelah filter tahun {TAHUN_BATAS_ATAS} ke bawah, sisa komentar: {len(df_scraped_filtered)}.")
    if len(df_scraped_filtered) == 0:
        print("[PERINGATAN] Tidak ada komentar dalam 1000 ulasan terbaru yang berasal dari tahun tersebut.")

else:
    print("\n[ERROR] Data scraping kosong, tidak dapat melakukan filtering.")


# --- 4. TAMPILAN OUTPUT LENGKAP (RATA KIRI DAN RAPI DENGAN TEXT WRAPPING) ---

if not df_scraped_filtered.empty:

    # Menghapus pengaturan Pandas yang tidak efektif untuk perataan konsol
    pd.reset_option('all')

    # --- Penyiapan Format Tabel Manual untuk Rata Kiri ---

    # Mendefinisikan lebar kolom:
    WIDTH_NO = 4
    WIDTH_USER = 20
    WIDTH_DATE = 20
    WIDTH_SCORE = 5
    WIDTH_THUMBS = 5

    # Lebar total untuk kolom selain content:
    # (3 spasi | 20 spasi | 3 spasi | 20 spasi | 3 spasi) = 49 spasi
    WIDTH_NON_CONTENT = WIDTH_NO + WIDTH_USER + WIDTH_DATE + WIDTH_SCORE + WIDTH_THUMBS + 10 # Total spasi dan pemisah: ~64

    # Lebar kolom konten untuk wrapping
    # Sisa lebar layar (misalnya 120) dikurangi lebar non-konten.
    WIDTH_CONTENT_WRAP = 120 - WIDTH_NON_CONTENT - 4 # Sekitar 50-55 karakter per baris wrap

    # Header Tabel
    header = (
        f"{'NO':<{WIDTH_NO}} | {'USERNAME':<{WIDTH_USER}} | {'AT':<{WIDTH_DATE}} | "
        f"{'CONTENT (Casefolded, Full)':<{WIDTH_CONTENT_WRAP}} | {'SCORE':<{WIDTH_SCORE}} | {'SUKA':<{WIDTH_THUMBS}} |"
    )
    separator = "=" * len(header)

    # String kosong yang akan digunakan sebagai indentasi untuk baris lanjutan konten
    INDENT_SPACER = " " * (WIDTH_NO + 1 + 1 + WIDTH_USER + 1 + 1 + WIDTH_DATE + 1 + 1) # Total 50 spasi sebelum konten

    # --- Mencetak Output ke Konsol ---

    print("\n\n=======================================================")
    print(f"HASIL AKHIR SCRAPING (Total Komentar Tampil: {len(df_scraped_filtered)})")
    print("=======================================================")
    print("CATATAN: Komentar LENGKAP ditampilkan dengan Text Wrapping Rata Kiri.")
    print("=======================================================")
    print("CATATAN: Teks komentar ('content') adalah data yang sudah melakukan casefolding.")
    print("---------------------------------------------------------------------------------------------------------------------------")

    # Cetak Header dan Separator
    print(header)
    print(separator)

    # Cetak Isi Tabel (Looping dan Text Wrapping)
    for index, row in df_scraped_filtered.iterrows():
        # 1. Terapkan Casefolding
        content_casefolded = str(row['content']).lower()

        # 2. Pecah konten panjang menjadi list baris
        wrapped_lines = textwrap.wrap(content_casefolded, width=WIDTH_CONTENT_WRAP)

        if not wrapped_lines:
            # Handle jika konten kosong
            wrapped_lines = ["-"]

        # --- Cetak Baris Pertama (dengan semua data) ---
        line = (
            f"{row['NO']:<{WIDTH_NO}} | {row['userName']:<{WIDTH_USER}} | {row['at']:<{WIDTH_DATE}} | "
            f"{wrapped_lines[0]:<{WIDTH_CONTENT_WRAP}} | {row['score']:<{WIDTH_SCORE}} | {row['thumbsUpCount']:<{WIDTH_THUMBS}} |"
        )
        print(line)

        # --- Cetak Baris Lanjutan (dengan indentasi) ---
        for subsequent_line in wrapped_lines[1:]:
            # Format baris lanjutan: menggunakan spasi kosong untuk kolom non-konten
            # dan memastikan konten lanjutan tetap Rata Kiri

            # Kolom Non-Content diganti dengan spasi kosong
            blank_columns = (
                f"{'':<{WIDTH_NO}} | {'':<{WIDTH_USER}} | {'':<{WIDTH_DATE}} | "
            )

            # Baris lanjutan hanya berisi indentasi, konten, dan spasi kosong di kolom akhir
            continuation_line = (
                f"{blank_columns}{subsequent_line:<{WIDTH_CONTENT_WRAP}} | {'':<{WIDTH_SCORE}} | {'':<{WIDTH_THUMBS}} |"
            )
            print(continuation_line)

    print(separator)


TARGET: Scraping 1000 komentar terbaru dari Growtopia
FILTER: Komentar dengan tanggal <= 31 Desember 2025

[INFO] Scraping berhasil! Total komentar mentah yang didapatkan: 1000.
[INFO] Setelah filter tahun 2025 ke bawah, sisa komentar: 1000.


HASIL AKHIR SCRAPING (Total Komentar Tampil: 1000)
CATATAN: Komentar LENGKAP ditampilkan dengan Text Wrapping Rata Kiri.
CATATAN: Teks komentar ('content') adalah data yang sudah melakukan casefolding.
---------------------------------------------------------------------------------------------------------------------------
NO   | USERNAME             | AT                   | CONTENT (Casefolded, Full)                           | SCORE | SUKA  |
1    | RE 8D                | 2025-10-03 08:56:35  | bad game, hard to log in, makes you lazy to play but | 3     | 0     |
     |                      |                      | if you want to play you definitely can't log in, no  |       |       |
     |                      |                      | idea 

/tmp/ipython-input-2496976504.py:84: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/tmp/ipython-input-2496976504.py:84: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')
